In [ ]:
from torchvision import dataset
from torchvision import transforms

from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report

import torchvision.models as models
import torchvision.nn as nn
import torch

In [ ]:
IMAGE_SIZE = 224
NORMALIZE_MEAN = [0.485, 0.456, 0.406]
NORMALIZE_STD = [0.229, 0.224, 0.225]
BATCH_SIZE = 16
NUM_WORKERS = 2
NUM_CLASSES = 9

test_transform = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.14)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.Normalize(NORMALIZE_MEAN, NORMALIZE_STD),
    transforms.ToTensor(),
])

test_dataset_path = '../../../dataset/test'
test_dataset = datasets.ImageFolder(root=test_dataset_path, transform=test_transform)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

In [ ]:
def test_model(model, loader, criterion, device):
    model.eval()
    test_loss, correct, total = 0.0, 0, 0

    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)

            preds = outputs.argmax(dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = test_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
model = models.mobilenet_v3_large(weights=None)

for param in model.features.parameters():
    param.requires_grad = False

old_linear = model.classifier[3]
in_feats = old_linear.in_features
classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(in_feats, num_classes)
)
model.classifier[3] = classifier

trained_model_path = "./mobilenetv3-05202025213410.pth"
model.load_state_dict(torch.load(trained_model_path))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

criterion = torch.nn.CrossEntropyLoss()
test_loss, test_accuracy = test_model(model, test_loader, criterion, device)
printf(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")